# Example of wet gas compression simulation

In [1]:
from neqsim.thermo.thermoTools import fluid_df, TPflash, printFrame
from neqsim.process import stream,  compressor
import pandas as pd

In [2]:
gascondensate = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane", "n-hexane", "C7", "C8", "C9", "C10", "C11", "C12", "C13", "C14", "C15", "C16", "C17", "C18", "C19", "C20"], 
        'MolarComposition[-]':  [0.53, 3.3, 72.98, 7.68, 4.1, 0.7, 1.42, 0.54, 0.67, 0.85, 1.33, 1.33, 0.78, 0.61, 0.42, 0.33, 0.42, 0.24, 0.3, 0.17, 0.21, 0.15, 0.15, 0.1], 
        'MolarMass[kg/mol]': [None,None, None,None,None,None,None,None,None,None,0.0913, 0.1041, 0.1188, 0.136, 0.150, 0.164, 0.179, 0.188, 0.204, 0.216, 0.236, 0.253, 0.27, 0.391],
        'RelativeDensity[-]': [None,None, None,None,None,None,None,None,None,None, 0.746, 0.768, 0.79, 0.787, 0.793, 0.804, 0.817, 0.83, 0.835, 0.843, 0.837, 0.84, 0.85, 0.877]
  } 

gascondensatedf = pd.DataFrame(gascondensate) 
print(gascondensatedf.head(30).to_string())
gascondensateFluid = fluid_df(gascondensatedf, lastIsPlusFraction=False)

   ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0       nitrogen                 0.53                NaN                 NaN
1            CO2                 3.30                NaN                 NaN
2        methane                72.98                NaN                 NaN
3         ethane                 7.68                NaN                 NaN
4        propane                 4.10                NaN                 NaN
5       i-butane                 0.70                NaN                 NaN
6       n-butane                 1.42                NaN                 NaN
7      i-pentane                 0.54                NaN                 NaN
8      n-pentane                 0.67                NaN                 NaN
9       n-hexane                 0.85                NaN                 NaN
10            C7                 1.33             0.0913               0.746
11            C8                 1.33             0.1041               0.768

In [3]:
TPflash(gascondensateFluid)
printFrame(gascondensateFluid)

                       0           1            2            3 4 5  \
0                              total          GAS          OIL       
1               nitrogen  5.33682E-3   5.66579E-3   8.76862E-6       
2                    CO2  3.32293E-2   3.52294E-2   8.34762E-4       
3                methane  7.34871E-1   7.80012E-1   3.74404E-3       
4                 ethane  7.73336E-2   8.19564E-2   2.46226E-3       
5                propane  4.12849E-2   4.35338E-2   4.86109E-3       
6               i-butane  7.04864E-3   7.35876E-3   2.02572E-3       
7               n-butane  1.42987E-2   1.48043E-2   6.10944E-3       
8              i-pentane  5.43752E-3   5.41878E-3   5.74109E-3       
9              n-pentane  6.74655E-3   6.57968E-3   9.44923E-3       
10              n-hexane  8.55906E-3    6.9988E-3   3.38294E-2       
11                 C7_PC  1.33924E-2   7.32828E-3   1.11609E-1       
12                 C8_PC  1.33924E-2    3.8988E-3   1.67153E-1       
13                 C

In [4]:
from neqsim.process import stream,  compressor, runProcess,clearProcess,GORfitter

pressure_inlet = 65.0 
temperature_inlet = 35.0

pressure_Out = 150.0
temperature_out = 99.0 

clearProcess()
inletStream  = stream('stream ', gascondensateFluid) 
inletStream.setTemperature(temperature_inlet, 'C')
inletStream.setPressure(pressure_inlet, 'bara')
inletStream.setFlowRate(5.0, 'MSm3/day')

GORfitter1 = GORfitter('GOR fitter 1', inletStream)
#GORfitter1.setGOR(2.0)
GORfitter1.setGVF(0.99)
GORfitter1.setReferenceConditions('actual')

compressor1 = compressor('compressor 1', GORfitter1.getOutStream())
compressor1.setOutletPressure(pressure_Out)
compressor1.setUsePolytropicCalc(True)
compressor1.setOutTemperature(temperature_out+273.15)
runProcess()

print('Feed GOR ', inletStream.getFluid().getPhase('gas').getCorrectedVolume()/inletStream.getFluid().getPhase('oil').getCorrectedVolume())
print('GOR fitted ', GORfitter1.getOutStream().getFluid().getPhase('gas').getCorrectedVolume()/GORfitter1.getOutStream().getFluid().getPhase('oil').getCorrectedVolume())
print('GVF fitted ', GORfitter1.getOutStream().getFluid().getPhase('gas').getCorrectedVolume()/(GORfitter1.getOutStream().getFluid().getPhase('oil').getCorrectedVolume()+GORfitter1.getOutStream().getFluid().getPhase('gas').getCorrectedVolume()))

print('Compressor power ', compressor1.getPower()/1e6, ' MW')
print('Polytropic efficiency ', compressor1.getPolytropicEfficiency() , ' [-]')
print('Polytropic fluid head ', compressor1.getPolytropicFluidHead(), ' kJ/kg')


Feed GOR  14.590220146656849
GOR fitted  98.99999999993761
GVF fitted  0.9899999999999938
Compressor power  8.647058772103753  MW
Polytropic efficiency  0.7962250986267035  [-]
Polytropic fluid head  94.24228389686442  kJ/kg
